### Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn import preprocessing
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
import plotly.express as px

import copy
import seaborn as sns
import os
from scipy import stats
import datetime

import sqlite3 as sqlite
from sqlalchemy import create_engine
%matplotlib inline

### Data

In [2]:
data_path = '/mnt/d/lighthouse/Midterm_data/'

In [3]:
# 15927485 rows × 42
flights = pd.read_csv(data_path + 'flights.csv', header = 0)

/home/david/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [16]:
# making duplicate so we dont have to re-read large csv files
flights_clean = copy.deepcopy(flights)

### Looking at Nan

In [17]:
total = flights_clean.isnull().sum().sort_values(ascending = False)
percent = (flights_clean.isnull().sum()/flights_clean.isnull().count()).sort_values(ascending = False)
missing_data = pd.concat([total,percent], axis=1, keys=['Total', 'Percent'])

In [6]:
missing_data.head(15)

,Total,Percent
no_name,15927485,1.000000
total_add_gtime,15818552,0.993161
longest_add_gtime,15818549,0.993161
first_dep_time,15818547,0.993160
cancellation_code,15658277,0.983098
late_aircraft_delay,12918001,0.811051
security_delay,12918001,0.811051
nas_delay,12918001,0.811051
weather_delay,12918001,0.811051
carrier_delay,12918001,0.811051


In [6]:
# dropping columsn with 80% missing AND [origin_city_name, dest_city_name] since we have [orgin, dest]
# dropping [cancelled, diverted, flights, dup] since they only contain one value
cols_to_drop_nan = list(missing_data[missing_data['Percent'] > 0.8].index)
cols_to_drop_other = ['dep_delay','taxi_out','taxi_in', 'wheels_off', 'wheels_on', 'arr_delay', 'cancelled', 'diverted','actual_elapsed_time','air_time', 'flights', 'dup']
flights_clean.drop(cols_to_drop_nan, axis = 1, inplace = True)
flights_clean.drop(cols_to_drop_other, axis = 1, inplace = True)

### dropping all rows with NA

In [8]:
# lost 3% of data, no biggie
flights_clean.dropna(inplace = True)

In [9]:
# (15605076, 30)
flights_clean.shape

(15605076, 28)

### seperating into numerical and categorical variables

In [12]:
# 18 numerical features
numerical = list(flights_clean.dtypes[flights_clean.dtypes != 'object'].index)
# 8 categorical features (10 if include city and state)
categorical = list(flights_clean.dtypes[flights_clean.dtypes == 'object'].index)

### removing outlier (numerical)

In [11]:
for i in numerical:
    
    q75, q25 = np.percentile(flights_clean[i], [75 ,25])
    
    IQR = stats.iqr(flights_clean[i], axis = 0)
    lower = q25 - (1.5 * IQR)
    upper = q75 + (1.5 * IQR)
    
    flights_clean = flights_clean[ (lower < flights_clean[i]) & (flights_clean[i] < upper) ]

In [12]:
flights_clean.shape

(10770229, 28)

In [13]:
flights_categorical = copy.deepcopy(flights_clean[categorical])
flights_numerical = copy.deepcopy(flights_clean[numerical])

### Binning and removing categorical

In [14]:
flights_categorical.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,op_unique_carrier,tail_num,origin,origin_city_name,dest,dest_city_name
0,2019-03-25,WN,WN,WN,WN,N8540V,MCI,"Kansas City, MO",PHX,"Phoenix, AZ"
1,2019-03-25,WN,WN,WN,WN,N8656B,MCI,"Kansas City, MO",PHX,"Phoenix, AZ"
2,2019-03-25,WN,WN,WN,WN,N8583Z,MCI,"Kansas City, MO",PHX,"Phoenix, AZ"
3,2019-03-25,WN,WN,WN,WN,N737JW,MCI,"Kansas City, MO",RDU,"Raleigh/Durham, NC"
4,2019-03-25,WN,WN,WN,WN,N705SW,MCI,"Kansas City, MO",RSW,"Fort Myers, FL"


In [15]:
#mkt_unq_carrier = ['NK', 'AS', 'G4', 'HA', 'B6', 'F9', 'VX']
# binning mkt_unique_carrier into 5 categories (WN, DL, UA, AA, others)
flights_categorical['bin_mkt_unique_carrier'] = np.where(flights_categorical['mkt_unique_carrier']=='WN',0,
                                                (np.where(flights_categorical['mkt_unique_carrier']=='DL',1,
                                                (np.where(flights_categorical['mkt_unique_carrier']=='UA',2,
                                                (np.where(flights_categorical['mkt_unique_carrier']=='AA',3,
                                                (np.where(np.isin(flights_categorical['mkt_unique_carrier'],mkt_unq_carrier),4,flights_categorical['mkt_unique_carrier'])))))))))

In [16]:
#branded_code = ['NK', 'AS_CODESHARE', 'G4', 'HA', 'B6', 'AS', 'HA_CODESHARE', 'F9', 'VX']
# binning branded_code_share into 8 categories (WN, DL_CODESHARE, UA, UA_CODESHARE, AA_CODESHARE, AA, DL, and others)
flights_categorical['bin_branded_code_share'] = np.where(flights_categorical['branded_code_share']=='WN',0,
                                                (np.where(flights_categorical['branded_code_share']=='DL_CODESHARE',1,
                                                (np.where(flights_categorical['branded_code_share']=='UA',2,
                                                (np.where(flights_categorical['branded_code_share']=='UA_CODESHARE',3,
                                                (np.where(flights_categorical['branded_code_share']=='AA_CODESHARE',4,
                                                (np.where(flights_categorical['branded_code_share']=='AA',5,
                                                (np.where(flights_categorical['branded_code_share']=='DL',6,
                                                (np.where(np.isin(flights_categorical['branded_code_share'],branded_code),7,flights_categorical['branded_code_share'])
                                                ))))))))))))))

In [17]:
#mkt_carrier = ['NK', 'AS', 'G4', 'HA', 'B6', 'F9', 'VX']
# binning mkt_carrier into 5 categories (WN, DL, UA, AA, others)
flights_categorical['bin_mkt_carrier'] = np.where(flights_categorical['mkt_carrier']=='WN',0,
                                                (np.where(flights_categorical['mkt_carrier']=='DL',1,
                                                (np.where(flights_categorical['mkt_carrier']=='UA',2,
                                                (np.where(flights_categorical['mkt_carrier']=='AA',3,
                                                (np.where(np.isin(flights_categorical['mkt_carrier'],mkt_carrier),4,flights_categorical['mkt_carrier'])))))))))

In [18]:
#op_unique_1 = ['YX', '9E', 'MQ', 'OH', 'YV', 'B6', 'EV', 'AS']
#op_unique_2 = ['NK', 'G4','HA', 'PT', 'CP', 'C5', 'ZW', 'AX', 'G7', 'EM', 'F9', 'QX', 'KS', 'VX', '9K']
# binning branded_code_share into 8 categories (WN, DL_CODESHARE, UA, UA_CODESHARE, AA_CODESHARE, AA, DL, and others)
flights_categorical['bin_op_unique_carrier'] = np.where(flights_categorical['op_unique_carrier']=='WN',0,
                                                (np.where(flights_categorical['op_unique_carrier']=='OO',1,
                                                (np.where(flights_categorical['op_unique_carrier']=='UA',2,
                                                (np.where(flights_categorical['op_unique_carrier']=='AA',3,
                                                (np.where(flights_categorical['op_unique_carrier']=='DL',4,
                                                (np.where(np.isin(flights_categorical['op_unique_carrier'],op_unique_1),5,
                                                (np.where(np.isin(flights_categorical['op_unique_carrier'],op_unique_2),6,flights_categorical['op_unique_carrier'])
                                                ))))))))))))

### before and after count plots

In [20]:
#fig, ax = plt.subplots(4, figsize=(16,16))

#sns.countplot(flights_categorical['mkt_unique_carrier'], ax = ax[0])
#sns.countplot(flights_categorical['branded_code_share'], ax = ax[1])
#sns.countplot(flights_categorical['mkt_carrier'], ax = ax[2])
#sns.countplot(flights_categorical['op_unique_carrier'], ax = ax[3])
#plt.savefig('categorical_count_plot.png')

In [21]:
#fig, ax = plt.subplots(4, figsize=(16,10))

#sns.countplot(flights_categorical['bin_mkt_unique_carrier'], ax = ax[0])
#sns.countplot(flights_categorical['bin_branded_code_share'], ax = ax[1])
#sns.countplot(flights_categorical['bin_mkt_carrier'], ax = ax[2])
#sns.countplot(flights_categorical['bin_op_unique_carrier'], ax = ax[3])

#plt.savefig('flights_bin_categorical_count_plot.png')

In [19]:
# dropping old columns ['mkt_unique_carrier','branded_code_share','mkt_carrier','op_unique_carrier']
#flights_categorical.drop(['mkt_unique_carrier','branded_code_share','mkt_carrier','op_unique_carrier'], axis =1, inplace = True)

### Seperating date time

In [15]:
# Seperating date to (YEAR, MONTH, DAY)
flights_categorical['fl_date'] = pd.to_datetime(flights_categorical['fl_date'])
flights_categorical['year'] = flights_categorical['fl_date'].dt.year
flights_categorical['month'] = flights_categorical['fl_date'].dt.month
flights_categorical['date'] = flights_categorical['fl_date'].dt.day
# Dropping the old date column
flights_categorical.drop('fl_date', axis = 1, inplace = True)

### Seperating into city and state

In [4]:
flights_categorical = pd.read_csv('/mnt/d/lighthouse/Midterm_data/flights_categorical_with_state.csv')
flights_numerical = pd.read_csv('/mnt/d/lighthouse/Midterm_data/flights_numerical.csv')
flights_categorical.drop('Unnamed: 0', axis = 1, inplace = True)
flights_numerical.drop('Unnamed: 0', axis = 1, inplace = True)

In [17]:
flights_categorical

,mkt_unique_carrier,branded_code_share,mkt_carrier,op_unique_carrier,tail_num,origin,origin_city_name,dest,dest_city_name,year,month,date
0,WN,WN,WN,WN,N8540V,MCI,"Kansas City, MO",PHX,"Phoenix, AZ",2019,3,25
1,WN,WN,WN,WN,N8656B,MCI,"Kansas City, MO",PHX,"Phoenix, AZ",2019,3,25
2,WN,WN,WN,WN,N8583Z,MCI,"Kansas City, MO",PHX,"Phoenix, AZ",2019,3,25
3,WN,WN,WN,WN,N737JW,MCI,"Kansas City, MO",RDU,"Raleigh/Durham, NC",2019,3,25
4,WN,WN,WN,WN,N705SW,MCI,"Kansas City, MO",RSW,"Fort Myers, FL",2019,3,25
...,...,...,...,...,...,...,...,...,...,...,...,...
15927480,WN,WN,WN,WN,N7702A,MCI,"Kansas City, MO",MSY,"New Orleans, LA",2019,3,25
15927481,WN,WN,WN,WN,N7878A,MCI,"Kansas City, MO",OAK,"Oakland, CA",2019,3,25
15927482,WN,WN,WN,WN,N760SW,MCI,"Kansas City, MO",PDX,"Portland, OR",2019,3,25
15927483,WN,WN,WN,WN,N8542Z,MCI,"Kansas City, MO",PHX,"Phoenix, AZ",2019,3,25


In [18]:
test = copy.deepcopy(flights_categorical[['origin_city_name','dest_city_name']])

In [19]:
test[['origin_city','origin_state']] = test.origin_city_name.str.split(',',expand=True)
test[['dest_city','dest_state']] = test.dest_city_name.str.split(',',expand=True)

In [20]:
flights_categorical.drop(['origin_city_name', 'dest_city_name'], axis = 1, inplace = True)

In [21]:
flights_categorical['origin_city'] = test['origin_city']
flights_categorical['origin_state'] = test['origin_state']
flights_categorical['dest_city'] = test['dest_city']
flights_categorical['dest_state'] = test['dest_state']

In [22]:
flights_categorical

,mkt_unique_carrier,branded_code_share,mkt_carrier,op_unique_carrier,tail_num,origin,dest,year,month,date,origin_city,origin_state,dest_city,dest_state
0,WN,WN,WN,WN,N8540V,MCI,PHX,2019,3,25,Kansas City,MO,Phoenix,AZ
1,WN,WN,WN,WN,N8656B,MCI,PHX,2019,3,25,Kansas City,MO,Phoenix,AZ
2,WN,WN,WN,WN,N8583Z,MCI,PHX,2019,3,25,Kansas City,MO,Phoenix,AZ
3,WN,WN,WN,WN,N737JW,MCI,RDU,2019,3,25,Kansas City,MO,Raleigh/Durham,NC
4,WN,WN,WN,WN,N705SW,MCI,RSW,2019,3,25,Kansas City,MO,Fort Myers,FL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15927480,WN,WN,WN,WN,N7702A,MCI,MSY,2019,3,25,Kansas City,MO,New Orleans,LA
15927481,WN,WN,WN,WN,N7878A,MCI,OAK,2019,3,25,Kansas City,MO,Oakland,CA
15927482,WN,WN,WN,WN,N760SW,MCI,PDX,2019,3,25,Kansas City,MO,Portland,OR
15927483,WN,WN,WN,WN,N8542Z,MCI,PHX,2019,3,25,Kansas City,MO,Phoenix,AZ


### Encoding categorical where there are too many categories

In [ ]:
# converting category to numbers
#flights_categorical['tail_num'] = pd.factorize(flights_categorical['tail_num'])[0]
#flights_categorical['origin'] = pd.factorize(flights_categorical['origin'])[0]
#flights_categorical['dest'] = pd.factorize(flights_categorical['dest'])[0]

### merging numerical and categorical back together

In [23]:
flights_final = pd.concat([flights_numerical, flights_categorical], axis=1)
flights_final.reset_index(drop=True)

,mkt_carrier_fl_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,...,tail_num,origin,dest,year,month,date,origin_city,origin_state,dest_city,dest_state
0,2098,2098,13198,14107,1755,1753.0,-2.0,7.0,1800.0,1832.0,...,N8540V,MCI,PHX,2019,3,25,Kansas City,MO,Phoenix,AZ
1,2238,2238,13198,14107,2000,1955.0,-5.0,6.0,2001.0,2030.0,...,N8656B,MCI,PHX,2019,3,25,Kansas City,MO,Phoenix,AZ
2,2451,2451,13198,14107,540,541.0,1.0,7.0,548.0,617.0,...,N8583Z,MCI,PHX,2019,3,25,Kansas City,MO,Phoenix,AZ
3,2213,2213,13198,14492,1550,1729.0,99.0,7.0,1736.0,2032.0,...,N737JW,MCI,RDU,2019,3,25,Kansas City,MO,Raleigh/Durham,NC
4,2096,2096,13198,14635,1045,1331.0,166.0,7.0,1338.0,1659.0,...,N705SW,MCI,RSW,2019,3,25,Kansas City,MO,Fort Myers,FL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15605071,2189,2189,13198,13495,2055,2111.0,16.0,5.0,2116.0,2240.0,...,N7702A,MCI,MSY,2019,3,25,Kansas City,MO,New Orleans,LA
15605072,1291,1291,13198,13796,1200,1328.0,88.0,7.0,1335.0,1527.0,...,N7878A,MCI,OAK,2019,3,25,Kansas City,MO,Oakland,CA
15605073,2470,2470,13198,14057,920,915.0,-5.0,7.0,922.0,1049.0,...,N760SW,MCI,PDX,2019,3,25,Kansas City,MO,Portland,OR
15605074,1651,1651,13198,14107,1125,1123.0,-2.0,9.0,1132.0,1158.0,...,N8542Z,MCI,PHX,2019,3,25,Kansas City,MO,Phoenix,AZ


In [24]:
# exporting to csv file
flights_final.to_csv('/mnt/d/lighthouse/Midterm_data/flights_outliers_no_encode.csv')